In [1]:
from cif import cif

In [2]:
data, subjects, measures = cif.createDataFrameFromOECD(countries = ['USA'], dsname = 'MEI',subject=['BSEMFT02'], frequency = 'M')

Requesting URL http://stats.oecd.org/SDMX-JSON/data/MEI/USA.BSEMFT02..M/all
Data downloaded from https://stats.oecd.org/SDMX-JSON/data/MEI/USA.BSEMFT02..M/all?dimensionAtObservation=AllDimensions


In [40]:
import os
import requests as rq
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib as mpl
import statsmodels.tsa.x13 as smX13
import statsmodels.tsa.arima_model as smARIMA
#import statsmodels.tsa.statespace.sarimax as smSARIMAX
import statsmodels.tsa.filters.hp_filter as smHP
from dateutil.relativedelta import relativedelta
#from subprocess import call
#from pathlib import Path
#import numbers
import warnings

In [14]:
s ="usa"

In [50]:
"https://stats.oecd.org/sdmx-json/data/DP_LIVE/USA.CPI.TOT.AGRWTH.M/OECD?json-lang=en&dimensionAtObservation=allDimensions&startPeriod=1948-01&endPeriod=2019-05"

'https://stats.oecd.org/sdmx-json/data/DP_LIVE/USA.CPI.TOT.AGRWTH.M/OECD?json-lang=en&dimensionAtObservation=allDimensions&startPeriod=1948-01&endPeriod=2019-05'

In [122]:
tt=makeOECDRequest('DP_LIVE', [['USA'],['CPI'],['TOT'],['AGRWTH'],'M'], params = {'startTime': '', 'endTime': '', 'dimensionAtObservation': 'AllDimensions'}, root_dir = 'http://stats.oecd.org/SDMX-JSON/data')


Requesting URL http://stats.oecd.org/SDMX-JSON/data/DP_LIVE/USA.CPI.TOT.AGRWTH.M/all


In [133]:
len([['USA'],['CPI'],['TOT'],['AGRWTH'],'M'])

5

In [132]:
tt=makeOECDRequest('MEI', [['USA'],['BSEMFT02'],[''],'M'], params = {'startTime': '', 'endTime': '', 'dimensionAtObservation': 'AllDimensions'}, root_dir = 'http://stats.oecd.org/SDMX-JSON/data')


Requesting URL http://stats.oecd.org/SDMX-JSON/data/MEI/USA.BSEMFT02..M/all


In [135]:
tt1 = tt.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [124]:
#tt1

In [125]:
obsList = tt1.get('dataSets')[0].get('observations')#['series']#['action']#['observations']

In [126]:
timeList = [item for item in tt1.get('structure').get('dimensions').get('observation') if item['id'] == 'TIME_PERIOD'][0]['values']


In [127]:
timeList

[{'id': '1956-01', 'name': 'Jan-1956'},
 {'id': '1956-02', 'name': 'Feb-1956'},
 {'id': '1956-03', 'name': 'Mar-1956'},
 {'id': '1956-04', 'name': 'Apr-1956'},
 {'id': '1956-05', 'name': 'May-1956'},
 {'id': '1956-06', 'name': 'Jun-1956'},
 {'id': '1956-07', 'name': 'Jul-1956'},
 {'id': '1956-08', 'name': 'Aug-1956'},
 {'id': '1956-09', 'name': 'Sep-1956'},
 {'id': '1956-10', 'name': 'Oct-1956'},
 {'id': '1956-11', 'name': 'Nov-1956'},
 {'id': '1956-12', 'name': 'Dec-1956'},
 {'id': '1957-01', 'name': 'Jan-1957'},
 {'id': '1957-02', 'name': 'Feb-1957'},
 {'id': '1957-03', 'name': 'Mar-1957'},
 {'id': '1957-04', 'name': 'Apr-1957'},
 {'id': '1957-05', 'name': 'May-1957'},
 {'id': '1957-06', 'name': 'Jun-1957'},
 {'id': '1957-07', 'name': 'Jul-1957'},
 {'id': '1957-08', 'name': 'Aug-1957'},
 {'id': '1957-09', 'name': 'Sep-1957'},
 {'id': '1957-10', 'name': 'Oct-1957'},
 {'id': '1957-11', 'name': 'Nov-1957'},
 {'id': '1957-12', 'name': 'Dec-1957'},
 {'id': '1958-01', 'name': 'Jan-1958'},


In [118]:
timeList

[{'id': '1956-01', 'name': 'Jan-1956'},
 {'id': '1956-02', 'name': 'Feb-1956'},
 {'id': '1956-03', 'name': 'Mar-1956'},
 {'id': '1956-04', 'name': 'Apr-1956'},
 {'id': '1956-05', 'name': 'May-1956'},
 {'id': '1956-06', 'name': 'Jun-1956'},
 {'id': '1956-07', 'name': 'Jul-1956'},
 {'id': '1956-08', 'name': 'Aug-1956'},
 {'id': '1956-09', 'name': 'Sep-1956'},
 {'id': '1956-10', 'name': 'Oct-1956'},
 {'id': '1956-11', 'name': 'Nov-1956'},
 {'id': '1956-12', 'name': 'Dec-1956'},
 {'id': '1957-01', 'name': 'Jan-1957'},
 {'id': '1957-02', 'name': 'Feb-1957'},
 {'id': '1957-03', 'name': 'Mar-1957'},
 {'id': '1957-04', 'name': 'Apr-1957'},
 {'id': '1957-05', 'name': 'May-1957'},
 {'id': '1957-06', 'name': 'Jun-1957'},
 {'id': '1957-07', 'name': 'Jul-1957'},
 {'id': '1957-08', 'name': 'Aug-1957'},
 {'id': '1957-09', 'name': 'Sep-1957'},
 {'id': '1957-10', 'name': 'Oct-1957'},
 {'id': '1957-11', 'name': 'Nov-1957'},
 {'id': '1957-12', 'name': 'Dec-1957'},
 {'id': '1958-01', 'name': 'Jan-1958'},


In [119]:
subjectList = tt1.get('structure').get('dimensions').get('observation')[1]['values']
measureList = tt1.get('structure').get('dimensions').get('observation')[2]['values']

In [129]:
obs = pd.DataFrame(obsList).transpose()
obs.rename(columns = {0: 'series'}, inplace = True)
obs['id'] = obs.index
obs = obs[['id', 'series']]
obs['dimensions'] = obs.apply(lambda x: re.findall('\d+', x['id']), axis = 1)
obs['subject'] = obs.apply(lambda x: subjectList[int(x['dimensions'][1])]['id'], axis = 1)
obs['measure'] = obs.apply(lambda x: measureList[int(x['dimensions'][2])]['id'], axis = 1)
obs['time'] = obs.apply(lambda x: timeList[int(x['dimensions'][5])]['id'], axis = 1)

In [130]:
obs.index

Index(['0:0:0:0:0:0', '0:0:0:0:0:1', '0:0:0:0:0:2', '0:0:0:0:0:3',
       '0:0:0:0:0:4', '0:0:0:0:0:5', '0:0:0:0:0:6', '0:0:0:0:0:7',
       '0:0:0:0:0:8', '0:0:0:0:0:9',
       ...
       '0:0:0:0:0:751', '0:0:0:0:0:752', '0:0:0:0:0:753', '0:0:0:0:0:754',
       '0:0:0:0:0:755', '0:0:0:0:0:756', '0:0:0:0:0:757', '0:0:0:0:0:758',
       '0:0:0:0:0:759', '0:0:0:0:0:760'],
      dtype='object', length=761)

In [131]:
obs.pivot_table(index = 'time', columns = ['subject', 'measure'], values = 'series')

subject,BSEMFT02
measure,STSA
time,
1956-01,0.374532
1956-02,0.374532
1956-03,0.374532
1956-04,0.749064
1956-05,1.123595
1956-06,1.872659
1956-07,2.238806
1956-08,1.865672


In [104]:
obs.head()

,id,series,dimensions,subject,measure,time
0:0:0:0:0:0,0:0:0:0:0:0,0.374532,"[0, 0, 0, 0, 0, 0]",CPI,TOT,1956-01
0:0:0:0:0:1,0:0:0:0:0:1,0.374532,"[0, 0, 0, 0, 0, 1]",CPI,TOT,1956-01
0:0:0:0:0:2,0:0:0:0:0:2,0.374532,"[0, 0, 0, 0, 0, 2]",CPI,TOT,1956-01
0:0:0:0:0:3,0:0:0:0:0:3,0.749064,"[0, 0, 0, 0, 0, 3]",CPI,TOT,1956-01
0:0:0:0:0:4,0:0:0:0:0:4,1.123595,"[0, 0, 0, 0, 0, 4]",CPI,TOT,1956-01


In [ ]:
def createOneCountryDataFrameFromOECD(country = 'CZE', dsname = 'MEI', subject = [], measure = [], frequency = 'M', startDate = None, endDate = None):      
    
    # Data download
    
    response = makeOECDRequest(dsname
                                 , [[country], subject, measure, [frequency]]
                                 , {'startTime': startDate, 'endTime': endDate, 'dimensionAtObservation': 'AllDimensions'})
    
    # Data transformation
    
    if (response.status_code == 200):
        
        responseJson = response.json()
        
        obsList = responseJson.get('dataSets')[0].get('observations')
        
        if (len(obsList) > 0):
            
            if (len(obsList) >= 999999):
                print('Warning: You are near response limit (1 000 000 observations).')
        
            print('Data downloaded from %s' % response.url)
            
            timeList = [item for item in responseJson.get('structure').get('dimensions').get('observation') if item['id'] == 'TIME_PERIOD'][0]['values']
            #subjectList = [item for item in responseJson.get('structure').get('dimensions').get('observation') if item['id'] == 'SUBJECT'][0]['values']
            #measureList = [item for item in responseJson.get('structure').get('dimensions').get('observation') if item['id'] == 'MEASURE'][0]['values']
            subjectList = responseJson.get('structure').get('dimensions').get('observation')[1]['values']
            measureList = responseJson.get('structure').get('dimensions').get('observation')[2]['values']
            
            obs = pd.DataFrame(obsList).transpose()
            obs.rename(columns = {0: 'series'}, inplace = True)
            obs['id'] = obs.index
            obs = obs[['id', 'series']]
            obs['dimensions'] = obs.apply(lambda x: re.findall('\d+', x['id']), axis = 1)
            obs['subject'] = obs.apply(lambda x: subjectList[int(x['dimensions'][1])]['id'], axis = 1)
            obs['measure'] = obs.apply(lambda x: measureList[int(x['dimensions'][2])]['id'], axis = 1)
            obs['time'] = obs.apply(lambda x: timeList[int(x['dimensions'][4])]['id'], axis = 1)
            #obs['names'] = obs['subject'] + '_' + obs['measure']
            
            #data = obs.pivot_table(index = 'time', columns = ['names'], values = 'series')
            
            data = obs.pivot_table(index = 'time', columns = ['subject', 'measure'], values = 'series')
            
            return(data, pd.DataFrame(subjectList), pd.DataFrame(measureList))
        
        else:
        
            print('Error: No available records, please change parameters')
            return(pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

    else:
        
        print('Error: %s' % response.status_code)
        return(pd.DataFrame(), pd.DataFrame(), pd.DataFrame())

In [59]:
tt.json()['structure']#['observation']#['dataSets']

{'links': [{'href': 'https://stats.oecd.org/SDMX-JSON/dataflow/DP_LIVE/all',
   'rel': 'dataflow'}],
 'name': 'OECD Data Live dataset',
 'description': 'OECD Data Live dataset',
 'dimensions': {'series': [{'keyPosition': 0,
    'id': 'LOCATION',
    'name': 'Country',
    'values': [{'id': 'USA', 'name': 'United States'}],
    'role': 'REF_AREA'},
   {'keyPosition': 1,
    'id': 'INDICATOR',
    'name': 'Indicator',
    'values': [{'id': 'CPI', 'name': 'Inflation (CPI)'}]},
   {'keyPosition': 2,
    'id': 'SUBJECT',
    'name': 'Subject',
    'values': [{'id': 'TOT', 'name': 'Total'}]},
   {'keyPosition': 3,
    'id': 'MEASURE',
    'name': 'Measure',
    'values': [{'id': 'AGRWTH', 'name': 'Annual growth rate (%)'}]},
   {'keyPosition': 4,
    'id': 'FREQUENCY',
    'name': 'Frequency',
    'values': [{'id': 'M', 'name': 'Monthly'}],
    'role': 'FREQ'}],
  'observation': [{'id': 'TIME_PERIOD',
    'name': 'Time',
    'values': [{'id': '1956-01', 'name': 'Jan-1956'},
     {'id': '1956

In [56]:
tt.json()['structure']['observation']

{'links': [{'href': 'https://stats.oecd.org/SDMX-JSON/dataflow/DP_LIVE/OECD',
   'rel': 'dataflow'}],
 'name': 'OECD Data Live dataset',
 'description': 'OECD Data Live dataset',
 'dimensions': {'series': [{'keyPosition': 0,
    'id': 'LOCATION',
    'name': 'Country',
    'values': [{'id': 'USA', 'name': 'United States'}],
    'role': 'REF_AREA'},
   {'keyPosition': 1,
    'id': 'INDICATOR',
    'name': 'Indicator',
    'values': [{'id': 'CPI', 'name': 'Inflation (CPI)'}]},
   {'keyPosition': 2,
    'id': 'SUBJECT',
    'name': 'Subject',
    'values': [{'id': 'TOT', 'name': 'Total'}]},
   {'keyPosition': 3,
    'id': 'MEASURE',
    'name': 'Measure',
    'values': [{'id': 'AGRWTH', 'name': 'Annual growth rate (%)'}]},
   {'keyPosition': 4,
    'id': 'FREQUENCY',
    'name': 'Frequency',
    'values': [{'id': 'M', 'name': 'Monthly'}],
    'role': 'FREQ'}],
  'observation': [{'id': 'TIME_PERIOD',
    'name': 'Time',
    'values': [{'id': '1956-01', 'name': 'Jan-1956'},
     {'id': '195

In [44]:
help(tt)

Help on Response in module requests.models object:

class Response(builtins.object)
 |  The :class:`Response <Response>` object, which contains a
 |  server's response to an HTTP request.
 |  
 |  Methods defined here:
 |  
 |  __bool__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if the status code of the response is between
 |      400 and 600 to see if there was a client error or a server error. If
 |      the status code, is between 200 and 400, this will return True. This
 |      is **not** a check to see if the response code is ``200 OK``.
 |  
 |  __enter__(self)
 |  
 |  __exit__(self, *args)
 |  
 |  __getstate__(self)
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __iter__(self)
 |      Allows you to use a response as an iterator.
 |  
 |  __nonzero__(self)
 |      Returns True if :attr:`status_code` is less than 400.
 |      
 |      This attribute checks if

In [ ]:
https://stats.oecd.org/SDMX-JSON/data/MEI/USA.BSEMFT02..M/all?dimensionAtObservation=AllDimensions

In [57]:
def makeOECDRequest(dsname, dimensions, params = None, root_dir = 'http://stats.oecd.org/SDMX-JSON/data'):
    
    """
    Make URL for the OECD API and return a response.
    
    Parameters
    -----
    dsname: str
        dataset identifier (e.g., MEI for main economic indicators)
    dimensions: list
        list of 4 dimensions (usually location, subject, measure, frequency)
    params: dict or None
        (optional) dictionary of additional parameters (e.g., startTime)
    root_dir: str
        default OECD API (https://data.oecd.org/api/sdmx-json-documentation/#d.en.330346)
        
    Returns
    -----
    results: requests.Response
        `Response <Response>` object
    
    """
    
    if not params:
        params = {}
    
    dim_args = ['+'.join(d) for d in dimensions]
    dim_str = '.'.join(dim_args)
    
    url = root_dir + '/' + dsname + '/' + dim_str + '/all'
    
    print('Requesting URL ' + url)
    return rq.get(url = url, params = params)